In [1]:
import os
%cd /home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
!pwd

/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
from src.MyDataset import MyDataset
from src.BertSentimentClassifier import BertSentimentClassifier
from src.utils import *
import warnings
import wandb
warnings.filterwarnings("ignore")

/root/anaconda3/envs/augllms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# download testing data (conception phase, unlabeled):
# !wget https://codalab.lisn.upsaclay.fr/my/datasets/download/26c12bc0-3878-4edf-8b4a-9682763c0b7e
# !unzip -o 26c12bc0-3878-4edf-8b4a-9682763c0b7e
# !rm 26c12bc0-3878-4edf-8b4a-9682763c0b7e

# load test data:
test_df = pd.read_csv('data/incidents_val.csv', index_col=0)

In [4]:
name_bert = "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"
tokenizer = BertTokenizer.from_pretrained(name_bert) 

In [5]:
label_encoder = LabelEncoder() 
label_encoder.classes_ = np.load('models/ST1/hazard_category/hazard_category_label_encoder.npy', allow_pickle=True)
device = "cuda"

In [6]:
label_encoder.classes_

array(['allergens', 'biological', 'chemical',
       'food additives and flavourings', 'foreign bodies', 'fraud',
       'migration', 'organoleptic aspects', 'other hazard',
       'packaging defect'], dtype=object)

In [7]:
def load_bert_model(weight_path, bert_model_name, num_classes):
    model_predict = BertSentimentClassifier(bert_model_name, num_classes)
    model_predict.load_state_dict(torch.load(weight_path, map_location=device))
    return model_predict 

In [8]:
max_len = 200
test_df = MyDataset(test_df, tokenizer, max_len, inference=True)
test_loader = DataLoader(test_df, batch_size=1, shuffle=True ) 
weight_path = "models/ST1/hazard_category/best_st1_hazard_category_large_200.pt"
model_predict = load_bert_model(weight_path, name_bert, num_classes=len(label_encoder.classes_)).to(device)


In [ ]:
predicted_labels_list = []
predicted_probs_list = []

for data in tqdm(test_loader):
    # move the inputs to the device
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)

    # Get predicted labels and probabilities
    # disable gradient computation
 
    outputs = model_predict(input_ids, attention_mask).to(device)
    predicted_labels = torch.argmax(outputs, dim=1).tolist()[0]
    predicted_probs = torch.nn.functional.softmax(outputs, dim=1)[0].tolist() 
    predicted_labels_list.append(predicted_labels)
    predicted_probs_list.append(predicted_probs) 


100%|██████████| 565/565 [00:08<00:00, 68.39it/s]


In [18]:
predicted_labels = []
predicted_probs = []

for data in tqdm(test_loader):
    # move the inputs to the device
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)

    # Get predicted labels and probabilities
    # disable gradient computation
 
    outputs = model_predict(input_ids, attention_mask).to(device)

    predicted_labels.append(label_encoder.inverse_transform(torch.argmax(outputs, dim=1).tolist())) 
    predicted_probs.append(torch.nn.functional.softmax(outputs, dim=1)[0].tolist())


100%|██████████| 565/565 [00:08<00:00, 69.59it/s]


In [ ]:
predictions = pd.DataFrame()

for column in ['hazard-category', 'product-category', 'hazard', 'product']:
  predictions[column] = predict(test_df.title.to_list(), f"bert_{column.replace('-', '_')}")

predictions.sample()